# ... add an affix-function to a standard name table during runtime?
If you are using the [standard name convention](../conventions/standard_name_convention.ipynb) you are familiar with transformation functions like "derivative_of_\<SN1\>_wrt_\<SN2\>" or "square_of_\<SN\>".

Now, you want to add your custom transformation function without changing the package code. Say, you want to add the transformation "maximum_of_<StdName>", referring to the maximum value of your data.

Here is the current state, which fails as expected:

In [2]:
import h5rdmtoolbox as h5tbx
from h5rdmtoolbox.conventions.standard_names.transformation import Transformation
from h5rdmtoolbox.conventions.standard_names import StandardName

snt = h5tbx.conventions.standard_names.StandardNameTable.from_zenodo(doi=8276716)

# check if the problem really exists:
try:
    snt['maximum_of_pressure']
except h5tbx.errors.StandardNameError as e:
    print(e)

"maximum_of_pressure" not found in Standard Name Table "Test".


## Implementing a new transformation function

We need to do two things:
 1. write a `function` which takes a regex result and generates the new standard name
 2. init a `Transformation` object which performs the regex match

The respective regex pattern to the above is `^maximum_of_(.*)$`.

The respective function is implemented in the following. It takes the result from a `re.match()` call and the standard name table object. The function then constructs a new standard name and returns it. Even if the regex match succeeded, it is still possible, that the function raises an error because the base standard_name may not exist.

In [3]:
def maximum_of(match, snt):
    # match is the result of `re.match(`^maximum_of_(.*)$, <user_input_value>)`
    groups = match.groups()
    assert len(groups) == 1
    sn = snt[groups[0]]
    new_description = f"Maximum of {sn.name}. {sn.description}"
    return StandardName(match.string, sn.units, new_description)

The actual transformation is managed by the class `Transformation`. It takes the above defined function and the respective regex pattern:

In [4]:
max_of = Transformation(r"^maximum_of_(.*)$", maximum_of)

We can already check if the pattern matching works:

In [5]:
max_of.match('maximum_static_pressure') is None

True

In [6]:
max_of.match('maximum_of_static_pressure') is None

False

## Add it to an existing standard name table during runtime:

In [7]:
snt.add_transformation(max_of)

In [8]:
snt['maximum_of_static_pressure']

<StandardName: "maximum_of_static_pressure" [Pa] Maximum of static_pressure. Static pressure refers to the force per unit area exerted by a fluid. Pressure is a scalar quantity.>

In [9]:
snt.transformations[-1] == max_of

True